In [5]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
import re
import json

class EDIParser:
    def __init__(self, message):
        """
        Inicializa la instancia con un mensaje EDI y lo parsea.
        """
        self.message = message
        self.segments = self._split_segments()
        self.parsed_message = self._parse_segments()

    def _split_segments(self):
        """
        Divide el mensaje EDI en segmentos usando el delimitador `'`.
        """
        return self.message.strip().split("'")

    def _parse_segments(self):
        """
        Parsea cada segmento en una lista de diccionarios.
        La clave es el identificador del segmento y el valor es una lista de elementos del segmento.
        """
        parsed_segments = []
        for segment in self.segments:
            if segment:
                elements = segment.split('+')
                segment_id = elements[0]
                if not elements[1:]:
                    raise ValueError(f"Missing data in segment: {segment_id}")
                segment_data = [self._split_elements(element) for element in elements[1:]]
                parsed_segments.append({segment_id: segment_data})
        return parsed_segments

    def _split_elements(self, element):
        """
        Divide los elementos en subelementos usando el delimitador `:`.
        """
        return element.split(':')

    def get_parsed_message(self):
        """
        Devuelve el mensaje parseado.
        """
        return self.parsed_message

    @staticmethod
    def parse_multiple_messages(messages):
        """
        Parsea múltiples mensajes EDI y maneja errores individualmente para cada mensaje.
        """
        parsed_messages = []
        for message in messages:
            try:
                parser = EDIParser(message)
                parsed_message = parser.get_parsed_message()
                parsed_messages.append(parsed_message)
            except Exception as e:
                # Manejo de errores: agregar el mensaje de error en lugar del mensaje parseado
                parsed_messages.append({"error": str(e), "message": message})
        return parsed_messages



In [21]:
# Ejemplo de uso
edi_messages = ["""
UNB+UNOA:2+ESB98261944:ZZ+ESA58898487:ZZ+240514:1627+8572999++IDADEP135COPAR'UNH+ADM_LLE8572999+COPARN:D:99A:UN:FTLM10'BGM+135+ESB98261944240514162751D1358572999+1'DTM+137:202405141627:203'FTX+ZSE+++VANESSA.MARTINEZ@MSC.COM'RFF+ACA:8572999'RFF+BN:048IBC0348981'TDT+20+YM420A+1++AP00051:::MSC+++9301328:::MSC DORINE:LR'LOC+14+08MBST'LOC+8+MACAS'DTM+133:202405172000:203'NAD+TR+A58898487+TERMINAL BEST'NAD+CN+B98261944+*MEDITERRANEAN SHIPPING COMPANY ESPANA, S.L.U.'GID+1+130:PK'FTX+AAA+++TESTLINER'MEA+WT+AAG+KGM:25000'SGP+8572999+130'EQD+CN++45G1'RFF+SQ:8572999'MEA+WT+AET+KGM:28970'UNT+20+ADM_LLE8572999'UNZ+1+IDADEP135COPAR'                
                
                """
]


In [22]:
# Parsear los mensajes EDI
parsed_messages = EDIParser.parse_multiple_messages(edi_messages)


In [23]:
import pprint
pprint.pprint(parsed_messages)

[[{'UNB': [['UNOA', '2'],
           ['ESB98261944', 'ZZ'],
           ['ESA58898487', 'ZZ'],
           ['240514', '1627'],
           ['8572999'],
           [''],
           ['IDADEP135COPAR']]},
  {'UNH': [['ADM_LLE8572999'], ['COPARN', 'D', '99A', 'UN', 'FTLM10']]},
  {'BGM': [['135'], ['ESB98261944240514162751D1358572999'], ['1']]},
  {'DTM': [['137', '202405141627', '203']]},
  {'FTX': [['ZSE'], [''], [''], ['VANESSA.MARTINEZ@MSC.COM']]},
  {'RFF': [['ACA', '8572999']]},
  {'RFF': [['BN', '048IBC0348981']]},
  {'TDT': [['20'],
           ['YM420A'],
           ['1'],
           [''],
           ['AP00051', '', '', 'MSC'],
           [''],
           [''],
           ['9301328', '', '', 'MSC DORINE', 'LR']]},
  {'LOC': [['14'], ['08MBST']]},
  {'LOC': [['8'], ['MACAS']]},
  {'DTM': [['133', '202405172000', '203']]},
  {'NAD': [['TR'], ['A58898487'], ['TERMINAL BEST']]},
  {'NAD': [['CN'],
           ['B98261944'],
           ['*MEDITERRANEAN SHIPPING COMPANY ESPANA, S.L.U.']]},
 